In [1]:
import hashlib
import io
import logging
import os
import pickle
import PIL.Image
from PIL import Image
from IPython.display import display
from IPython.display import clear_output
import random
import numpy as np

from object_detection.utils import dataset_util
from object_detection.utils import visualization_utils as vis_util

%run ./variables.ipynb
%run ./utils.ipynb

import tensorflow as tf
from tensorflow import keras
print('Tensorflow version: ', tf.__version__)
print("Eager execution: ", tf.executing_eagerly())
print("Listing CPUs: ", tf.config.list_physical_devices('CPU'))
print("Listing GPUs: ", tf.config.list_physical_devices('GPU'))
print("Listing XLA_GPUs: ", tf.config.list_physical_devices('XLA_GPU'))

Tensorflow version:  2.1.0
Eager execution:  True
Listing CPUs:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Listing GPUs:  []
Listing XLA_GPUs:  [PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]


In [15]:
def create_tf_example(annotation, id_map, binary=True, segmentation=False, verbose=False):
    ## LOADING IMAGE
    img_path = os.path.join(DATA, annotation["img_path"])
    encoded_image_data, width, height = load_png(img_path)
    key = hashlib.sha256(encoded_image_data).hexdigest()
    
    if verbose:
        full_image = np.array(Image.open(img_path))
        full_image = np.expand_dims(full_image, -1)
        full_image = np.repeat(full_image, 3, 2)
        full_image = np.array(full_image)
    
    ## GENERAL FEATURES
    image_format = IMG_TYPE.encode('utf8') # b'jpeg' or b'png'
    ## DEFINING BOUNDING BOXES
    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    labels = annotation["labels"]
    for label in labels:
        ymins.append(label["xmin"]/width)
        ymaxs.append(label["xmax"]/width)
        xmins.append(label["ymin"]/height)
        xmaxs.append(label["ymax"]/height)
        if verbose:
            vis_util.draw_bounding_box_on_image_array(full_image, 
                                             label["xmin"]/width, 
                                             label["ymin"]/height,
                                             label["xmax"]/width,
                                             label["ymax"]/height,
                                             use_normalized_coordinates=True)
        if binary:
            classes_text.append(next(iter(id_map)).encode('utf8'))
            classes.append(id_map[next(iter(id_map))])
        else:
            classes_text.append(label["taxon"].encode('utf8'))
            classes.append(id_map[label["taxon"]])
    
    feature_dict = {
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs)
    }
    
    if segmentation:
        masks_list = []
        for label in labels:
            path = label["mask_path"]
            im = Image.open(os.path.join(DATA, path))
            imgByteArr = io.BytesIO()
            im.save(imgByteArr, format='PNG')
            masks_list.append(imgByteArr.getvalue())
            if verbose:
                img_pil = Image.open(imgByteArr)
                img_pil = np.array(img_pil)
                vis_util.draw_mask_on_image_array(full_image,img_pil)
        feature_dict['image/object/mask']=dataset_util.bytes_list_feature(masks_list)
    
    if verbose:
        display(Image.fromarray(full_image))
        
    tf_example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return tf_example

In [16]:
def create_tf_record(annotations_path, id_map, output_path, binary=True, segmentation=False, verbose=False):
    writer = tf.io.TFRecordWriter(output_path)
    print("Creating tfrecord based on "+str(len(annotations_path))+" example(s). Save location: "+output_path)
    #print("0 %")
    a = display(str(0)+"/"+str(len(annotations_path)),display_id=True)
    for i, annotation_file in enumerate(annotations_path):
        a.update(str(i+1)+"/"+str(len(annotations_path)))
        annotation_path = os.path.join(DATA, ANNOTATIONS_FOLDER, annotation_file)
        if os.path.isfile(annotation_path):
            annotation = pickle.load(open(annotation_path, "rb"))
            tf_example = create_tf_example(annotation, id_map, binary=binary, segmentation=segmentation, verbose=verbose)
            writer.write(tf_example.SerializeToString())
        else:
            print("WARNING: Error while loading annoation "+annotation_path+". Ignoring image.")
    writer.close()
    print("Finished!")

In [17]:
def get_map(binary=True, save=True):
    if binary: pb_name = "binary_label_map"
    else: pb_name = "multiclass_label_map"
    id_map = pickle.load(open(os.path.join(DATA, MAPS_FOLDER, pb_name+".pickle"), "rb" ))
    if save: create_pbtxt(id_map, OUPUT_PATH+'/'+pb_name+".pbtxt")
    return id_map

In [18]:
# LISTING ALL ANNOTATIONS FROM THE IMAGE FOLDER
img_path = os.path.join(DATA, ANNOTATIONS_FOLDER)
annotations = []
for (dirpath, dirnames, filenames) in os.walk(img_path):
    for filename in filenames:
        splitted = filename.split('.')
        if len(splitted) and splitted[1].lower()=="pickle":
            annotations.append(filename)
random.shuffle(annotations)

In [19]:
# Parameters
binary_dataset = False
segmentation = False
verbose = False
# Get id_map
id_map = get_map(binary=binary_dataset, save=True)
# Train dataset
train_record_path = os.path.join(OUPUT_PATH, TRAIN_OUTPUT_FILE)
check_dirs(train_record_path)
create_tf_record(annotations[0:int(PERCENTAGE_TRAIN*len(annotations))], 
                 id_map,
                 train_record_path, 
                 binary=binary_dataset, 
                 segmentation=True, 
                 verbose=verbose)
# Test dataset
test_record_path = os.path.join(OUPUT_PATH, TEST_OUTPUT_FILE)
check_dirs(test_record_path)
create_tf_record(annotations[int(PERCENTAGE_TRAIN*len(annotations)):len(annotations)], 
                 id_map,
                 test_record_path, 
                 binary=binary_dataset, 
                 segmentation=True,
                 verbose=verbose)

Creating tfrecord based on 18000 example(s). Save location: /mnt/nvme-storage/pfauregi/artificial_datasets/dataset03_tfr/train_dataset.record


'18000/18000'

Finished!
Creating tfrecord based on 2001 example(s). Save location: /mnt/nvme-storage/pfauregi/artificial_datasets/dataset03_tfr/test_dataset.record


'2001/2001'

Finished!
